In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser 
from langchain_community.document_loaders.csv_loader import CSVLoader
import os  # 환경 변수 설정을 위한 모듈

# OpenAI API 키 설정 (실제 사용 시에는 안전한 방법으로 API 키를 관리해야 함)
os.environ["OPENAI_API_KEY"] = ""

In [2]:
# 1. CSV 파일 로드
# - CSVLoader를 사용하여 지정된 경로의 CSV 파일을 로드
# - 각 행(row)이 하나의 문서(Document)로 변환됨
loader = CSVLoader(file_path="../data/titanic.csv")
docs = loader.load()
print(f"Row 수: {len(docs)}")

Row 수: 891


In [3]:
# 2. 불러온 문서 내용 확인
# - 특정 문서의 내용을 출력하여 데이터 확인
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")


[페이지내용]
PassengerId: 11
Survived: 1
Pclass: 3
Name: Sandstrom, Miss. Marguerite Rut
Sex: female
Age: 4
SibSp: 1
Parch: 1
Ticket: PP 9549
Fare: 16.7
Cabin: G6
Embarked: S

[metadata]
{'source': '../data/titanic.csv', 'row': 10}



In [4]:
# 3. 문서를 작은 청크로 분할
# - RecursiveCharacterTextSplitter를 사용하여 텍스트를 일정 크기의 청크로 나눔
# - chunk_size: 한 개의 청크 크기 (문자 수)
# - chunk_overlap: 청크 간 겹치는 부분의 크기 (연속성 유지)
chunk_size = 1000  # 적절한 값 조절 필요
chunk_overlap = 100  # 적절한 값 조절 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# 문서를 여러 개의 작은 청크로 나눔
splits = text_splitter.split_documents(docs)
print(f"청크 개수: {len(splits)}")  # 생성된 청크 개수 확인

청크 개수: 891


In [5]:
# 4. 벡터스토어(Vector Store) 생성
# - FAISS(Facebook AI Similarity Search)를 사용하여 벡터 DB를 생성
# - OpenAIEmbeddings를 사용하여 텍스트를 벡터로 변환하여 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_45704\1849054279.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))


In [6]:
# 5. 문서에서 정보를 검색할 수 있도록 검색기(retriever) 생성
retriever = vectorstore.as_retriever()

In [7]:
# 6. 프롬프트 템플릿 정의 (출처 포함)
# - 검색된 문서를 기반으로 질문에 답변하도록 설계
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.
검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.
#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [11]:
# 7. LLM 모델 선택
# - OpenAI의 GPT-4o-mini 모델을 사용하여 검색된 정보를 바탕으로 응답을 생성
# - temperature=0 설정으로 일관된 답변을 생성하도록 설정
llm = ChatOpenAI(model_name="gpt-5-nano", temperature=1)

In [12]:
# 8. RAG(Retrieval-Augmented Generation) 체인 생성
# - 검색된 문서를 바탕으로 LLM이 답변을 생성하도록 연결
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # 검색을 수행하는 객체 입력
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "데이터에 대해서 알려주세요."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

다음은 주어진 컨텍스트의 데이터에 대한 요약입니다.

- 데이터 원천/형식
  - Titanic.csv에서 발췌된 승객 데이터로 보이며, 각 행은 한 명의 승객 정보를 담고 있습니다.
  - 각 레코드의 필드(열)는 일반적으로 PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked 로 구성되어 있습니다.
  - 출처 표기: source가 ../data/titanic.csv이고, 각 문서에 row 번호가 함께 제시되어 있습니다.

- 예시 레코드 요약 (주어진 4개 레코드)
  - Dakic, Mr. Branko
    - PassengerId: 688
    - Survived: 0
    - Pclass: 3
    - Sex: male
    - Age: 19
    - SibSp: 0
    - Parch: 0
    - Ticket: 349228
    - Fare: 10.1708
    - Cabin: (기재 없음)
    - Embarked: S
  - Hansen, Mr. Henry Damsgaard
    - PassengerId: 624
    - Survived: 0
    - Pclass: 3
    - Sex: male
    - Age: 21
    - SibSp: 0
    - Parch: 0
    - Ticket: 350029
    - Fare: 7.8542
    - Cabin: (기재 없음)
    - Embarked: S
  - Backstrom, Mr. Karl Alfred
    - PassengerId: 207
    - Survived: 0
    - Pclass: 3
    - Sex: male
    - Age: 32
    - SibSp: 1
    - Parch: 0
    - Ticket: 3101278
    - Fare: 15.85
    - Cabin: (기재 없음)
    - Embarked: S

In [14]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "이 데이터를 가지고 어떤 걸 분석할 수 있을지 아이디어 5개 리스트 형식으로 정리해줘."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

다음은 이 데이터를 가지고 할 수 있는 분석 아이디어 5가지입니다:

1) 생존 예측 모델 초안
- 목표 변수: Survived
- 피처: Pclass, Sex, Age, SibSp, Parch, Fare, Embarked 등을 활용해 로지스틱 회귀나 트리 기반 모델(RandomForest, GradientBoosting 등)로 예측 성능 비교.

2) 항목별 요약 통계 및 생존 비율 분석
- Pclass별 생존 비율, Sex별 생존 비율, Embarked별 생존 비율을 계산
- Fare의 분포(평균/중앙값/사분위)와 생존 여부와의 관계를 시각화

3) Fare와 Pclass의 상호작용 분석
- Fare가 생존에 미치는 영향이 Pclass에 따라 달라지는지 확인
- 상호작용 항을 포함한 모델(예: 로지스틱 회귀의 상호작용 항)이나 교차표를 통해 해석

4) Age 결측치 처리 전략과 영향 평가
- Age가 비어 있는 항목의 처리 방법 비교(Imputation: 평균/중앙값, 모델 기반 예측 등)
- 결측치를 처리한 후 생존 예측 모델의 성능 변화나 특정 그룹에서의 영향 비교

5) 가족 구성(SibSp, Parch)과 생존 관계 분석
- 형제자매/배우자 수(SibSp)와 부모/자식 수(Parch)가 생존에 미치는 영향 분석
- 가족 크기 합치 변수 생성(예: 가족 총합) 후 생존률 비교 및 모델에 반영

필요 시 각 아이디어에 대해 시각화(막대그래프, 상자그림, 히트맵)와 교차분석 표를 함께 제시하면 이해도가 높아집니다.
